<a href="https://colab.research.google.com/github/Shoumodipdc/Dental-College-Data-Extraction/blob/main/Dental_College_Data_Scrapper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
'''
!pip uninstall duckduckgo_search
!pip install ddgs

!pip install playwright

!playwright install chromium
!playwright install-deps chromium

!pip uninstall google-search-results
!pip install serpapi
!pip install json-repair
!pip install -U groq
!pip install "jupyter_client<8" \
    google_colab_selenium \
    google-search-results \
    groq \
    crawl4ai \
    duckduckgo_search
!pip install -U ddgs
'''

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 53.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 MB 18.9 MB/s eta 0:00:00
(node:1016) [DEP0169] DeprecationWarning: `url.parse()` behavior is not standardized and prone to errors that have security implications. Use the WHATWG URL API instead. CVEs are not issued for `url.parse()` vulnerabilities.
(Use `node --trace-deprecation ...` to show where the warning was created)
167.3 MiB [] 0% 339.3s167.3 MiB [] 0% 43.9s167.3 MiB [] 0% 18.5s167.3 MiB [] 0% 15.3s167.3 MiB [] 0% 11.1s167.3 MiB [] 1% 6.0s167.3 MiB [] 1% 5.3s167.3 MiB [] 2% 4.7s167.3 MiB [] 2% 4.4s167.3 MiB [] 3% 3.9s167.3 MiB [] 3% 4.0s167.3 MiB [] 4% 3.9s167.3 MiB [] 5% 3.6s167.3 MiB [] 6% 3.3s167.3 MiB [] 6% 3.2s167.3 MiB [] 7% 3.0s167.3 MiB [] 8% 2.9s167.3 MiB [] 9% 2.8s167.3 MiB [] 10% 2.8

In [1]:
#!pip install nest_asyncio

In [2]:
#pip install google-search-results

In [3]:
import os, json, asyncio, re, warnings, sys, logging
from groq import Groq
from ddgs import DDGS
from serpapi import GoogleSearch
from contextlib import redirect_stdout
import nest_asyncio
nest_asyncio.apply()
import urllib.parse
from groq import Groq
from ddgs import DDGS
from serpapi import GoogleSearch
from crawl4ai import AsyncWebCrawler, BrowserConfig, CrawlerRunConfig, CacheMode
from google.colab import userdata

# Silence warnings and logs globally
warnings.filterwarnings("ignore")
logging.getLogger("crawl4ai").setLevel(logging.ERROR)
logging.getLogger("playwright").setLevel(logging.ERROR)

In [4]:
# Retrieve the keys securely
os.environ["GROQ_API_KEY"] = userdata.get('GROQ_API_KEY')
os.environ["SERPAPI_KEY"] = userdata.get('SERPAPI_KEY')

groq_client = Groq(api_key=os.environ.get("GROQ_API_KEY"))

# Now you can use them in your scripts
print("API keys loaded into environment variables!")

API keys loaded into environment variables!


In [5]:
# -----------------------------
# 1. Google Maps fetch
# -----------------------------
def fetch_google_maps_data(college, state):
    query = f"{college}, {state}, India"
    query = urllib.parse.quote_plus(query)

    params = {
        "engine": "google_maps",
        "q": query,
        "type": "search",
        "hl": "en",
        "gl": "in",
        "api_key": os.environ.get("SERPAPI_KEY")
    }

    data = GoogleSearch(params).get_dict()

    # Fallback logic
    place = None
    if data.get("place_results"):
        place = data["place_results"]
    elif data.get("local_results"):
        candidates = data["local_results"]

        def score(p):
            s = 0
            title = p.get("title", "").lower()
            if "college" in title:
                s += 5
            if "hospital" in title and "college" not in title:
                s -= 5
            for w in college.lower().split():
                if w in title:
                    s += 1
            reviews = p.get("reviews", 0)
            if reviews and reviews < 5000:
                s += 3
            if reviews and reviews > 10000:
                s -= 3
            return s

        candidates.sort(key=score, reverse=True)
        place = candidates[0]

    return {
        "address": place.get("address") if place else None,
        "google_rating": place.get("rating") if place else None,
        "google_reviews": f"{place.get('reviews')} Google reviews"
          if place and place.get("reviews") else "0 Google reviews",
        "maps_link": f"https://www.google.com/maps/search/?api=1&query={query}"

    }



In [6]:
def fetch_college_logo(college):
    query = f'"{college}" official website logo'
    try:
        with DDGS() as ddgs:
            for r in ddgs.images(query, max_results=1):
                return r.get("image")
    except:
        pass
    return None


In [7]:

# -----------------------------
# 2. Main async function
# -----------------------------
async def get_accurate_college_profile(college, state):
    print(f"Starting audit: {college}")

    # 1. Google Maps
    google_data = fetch_google_maps_data(college, state)

    # 2. Strategic search URLs
    # # We force the search to look for the official site and DCI records
    queries = [
        f'"{college}" {state} dental college management university intake recognition',
        f'"{college}" {state} campus area faculty hospital associated',
        f'"{college}" {state} nearest airport railway station road connectivity',
        f'"{college}" {state} mds specialties list',
        f'"{college}" {state} official website',
        f'"{college}" {state} DCI recognition status 2024 2025',
        f'"{college}" total faculty list MDS specialties',
        f'"{college}" affiliated university'
    ]

    urls = []
    try:
      with DDGS() as ddgs:
        for q in queries:
          # Filter out Scribd and old PDF sites to prevent timeouts
          for r in ddgs.text(q, max_results=3):
            link = r["href"]
            if not any(x in link for x in ["scribd.com", "facebook.com", "youtube.com"]):
                urls.append(link)
      urls = list(set(urls))
    except:
        print("DDGS not installed or failed. Skipping strategic URLs.")

    # 3. Crawl4AI pages
    combined_markdown = ""
    try:
        from crawl4ai import AsyncWebCrawler, BrowserConfig, CrawlerRunConfig, CacheMode
        with redirect_stdout(open(os.devnull, "w")):
            async with AsyncWebCrawler(config=BrowserConfig(headless=True, verbose=False)) as crawler:
                tasks = [
                    crawler.arun(
                        url=u,
                        config=CrawlerRunConfig(
                            cache_mode=CacheMode.DISABLED,
                            page_timeout=20000
                        )
                    )
                    for u in urls
                ]
                pages = await asyncio.gather(*tasks)
                combined_markdown = " ".join(
                    p.markdown for p in pages if p and p.success and p.markdown
                )
    except:
        print("Crawl4AI not installed or failed. Skipping page scraping.")

    logo_link = fetch_college_logo(college)

    # 4. Groq audit
    print(f"Groq auditing: {college}")
    prompt = f"""
Extract verified academic and logistical data for:
{college}, {state}

CONTEXT:
{combined_markdown[:15000]}

STRICT RULES:
- ug_courses = 1 (BDS)
- pg_courses = integer MDS count
- Use null if unknown
- No hallucination
- Use official naming

RETURN JSON ONLY:
{{
  "management": "string",
  "recognised_by": "Dental Council of India",
  "status": "Recognised / Permitted",
  "total_intake": "string",
  "total_faculty": "string",
  "location_type": "Urban / Rural / Semi-Urban",
  "gender": "Co-education / Female Only",
  "total_area": "string",
  "hospital_associated": "string",
  "hospital_distance": "string",
  "nearest_airport": "Name and distance",
  "nearest_railway": "Name and distance",
  "road_connectivity": "string",
  "ug_courses": 1,
  "pg_courses": 0,
  "university": "string",
  "estd": 0000,
  "short_overview": "3 sentences",
  "long_overview": "7 sentences"
}}r
"""

    try:
        res = groq_client.chat.completions.create(
            model="llama-3.1-8b-instant",
            messages=[
                {"role": "system", "content": "You are a medical education auditor."},
                {"role": "user", "content": prompt}
            ],
            temperature=0,
            response_format={"type": "json_object"}
        )
        academic_info = json.loads(res.choices[0].message.content)
    except:
        print("Groq audit failed. Returning empty academic info.")
        academic_info = {}

    # 5. Merge final record
    final_record = {
        "name": college,
        "state": state,
        "clg_logo_image": logo_link,
        **google_data,
        **academic_info
    }

    print(f"Audit complete: {college}")
    return final_record




In [8]:
college_records = []

college = await get_accurate_college_profile(
    "Govt. Dental College & Hospital, Vijayawada",
    "Andhra Pradesh"
)

college_records.append(college)
college


Starting audit: Govt. Dental College & Hospital, Vijayawada


[FETCH]... ↓ https://www.orchidfoundation.info/college/govt-dental-college-hospital-vijayawada                    |
✓ | ⏱: 8.98s 

[SCRAPE].. ◆ https://www.orchidfoundation.info/college/govt-dental-college-hospital-vijayawada                    |
✓ | ⏱: 0.04s 

[COMPLETE] ● https://www.orchidfoundation.info/college/govt-dental-college-hospital-vijayawada                    |
✓ | ⏱: 9.12s 

[FETCH]... ↓ https://medicalneetug.com/college/government-dental-college-vijayawada                               |
✓ | ⏱: 16.55s 

...

[FETCH]... ↓ https://www.careers360.com/colleges/government-dental-college-and-hospital-vijayawada                |
✓ | ⏱: 33.90s 

[SCRAPE].. ◆ https://www.careers360.com/colleges/government-dental-college-and-hospital-vijayawada                |
✓ | ⏱: 0.39s 

[COMPLETE] ● https://www.careers360.com/colleges/government-dental-college-and-hospital-vijayawada                |
✓ | ⏱: 34.33s 

Groq auditing: Govt. Dental College & Hospital, Vijayawada
Audit complete: Govt. Dental College & Hospital, Vijayawada


{'name': 'Govt. Dental College & Hospital, Vijayawada',
 'state': 'Andhra Pradesh',
 'clg_logo_image': 'https://lookaside.fbsbx.com/lookaside/crawler/media/?media_id=100050841296178',
 'address': '6-3, 52-1, 6-3, ESI Rd, NTR Colony, Veterinary Colony, Gunadala, Kanuru, Andhra Pradesh 520004',
 'google_rating': 4,
 'google_reviews': '0 Google reviews',
 'maps_link': 'https://www.google.com/maps/search/?api=1&query=Govt.+Dental+College+%26+Hospital%2C+Vijayawada%2C+Andhra+Pradesh%2C+India',
 'management': 'Government',
 'recognised_by': 'Dental Council of India',
 'status': 'Recognised',
 'total_intake': '100',
 'total_faculty': '50',
 'location_type': 'Urban',
 'gender': 'Co-education',
 'total_area': '10 acres',
 'hospital_associated': 'Yes',
 'hospital_distance': '1 km',
 'nearest_airport': 'Vijayawada Airport, 10 km',
 'nearest_railway': 'Vijayawada Railway Station, 5 km',
 'road_connectivity': 'Good',
 'ug_courses': 1,
 'pg_courses': 0,
 'university': 'NTR University of Health Scien

In [ ]:
# EXECUTION
college_1 = await get_accurate_college_profile("Govt. Dental College & Hospital, Vijayawada", "Andhra Pradesh")
print(json.dumps(college_1, indent=2))
college_records.append(college_1)

In [9]:
# EXECUTION
college_2 = await get_accurate_college_profile("Govt. Dental College, RIMS, Kadapa", "Andhra Pradesh")
print(json.dumps(college_2, indent=2))
college_records.append(college_2)

Starting audit: Govt. Dental College, RIMS, Kadapa


[FETCH]... ↓ https://amcana.org/p/endaro-mahanubhavulu-details.html                                               |
✓ | ⏱: 14.64s 

[SCRAPE].. ◆ https://amcana.org/p/endaro-mahanubhavulu-details.html                                               |
✓ | ⏱: 6.31s 

[COMPLETE] ● https://amcana.org/p/endaro-mahanubhavulu-details.html                                               |
✓ | ⏱: 21.12s 

[FETCH]... ↓ https://www.careers360.com/colleges/government-dental-college-kadapa                                 |
✓ | ⏱: 26.84s 

...

[ERROR]... × https://dciindia.gov.in...ype=0&Status=--Select--  | Error: Unexpected error in _crawl_web at line 718
in _crawl_web (../usr/local/lib/python3.12/dist-packages/crawl4ai/async_crawler_strategy.py):
Error: Failed on navigating ACS-GOTO:
Page.goto: Timeout 20000ms exceeded.
Call log:
  - navigating to 
"https://dciindia.gov.in/CollegeSearch.aspx?ColName=&CourseId=2&SplId=0&StateId=&Hospital=&Type=0&Status=--Select--
", waiting until "domcontentloaded"


Code context:
 713                                   tag="GOTO",
 714                                   params={"url": url},
 715                               )
 716                               response = None
 717                           else:
 718 →                             raise RuntimeError(f"Failed on navigating ACS-GOTO:\n{str(e)}")
 719   
 720                       # ──────────────────────────────────────────────────────────────
 721                       # Walk the redirect chain.  Playwright returns only the last
 722                       # hop, so we trace the `request.redirected_from` links until the
 723                       # first response that differs from the final one and surface its 

[FETCH]... ↓ https://neetadmission.in/UG/college/1346/govt.-dental-college,-rims,-kadapa                          |
✓ | ⏱: 68.84s 

[SCRAPE].. ◆ https://neetadmission.in/UG/college/1346/govt.-dental-college,-rims,-kadapa                          |
✓ | ⏱: 0.10s 

[COMPLETE] ● https://neetadmission.in/UG/college/1346/govt.-dental-college,-rims,-kadapa                          |
✓ | ⏱: 68.95s 

Groq auditing: Govt. Dental College, RIMS, Kadapa
Audit complete: Govt. Dental College, RIMS, Kadapa
{
  "name": "Govt. Dental College, RIMS, Kadapa",
  "state": "Andhra Pradesh",
  "clg_logo_image": "http://www.gdchkadapa.com/images/logo-old-bg-removed.png",
  "address": "RIMS Medical College, Puttampalli, Andhra Pradesh 516002",
  "google_rating": 4.2,
  "google_reviews": "0 Google reviews",
  "maps_link": "https://www.google.com/maps/search/?api=1&query=Govt.+Dental+College%2C+RIMS%2C+Kadapa%2C+Andhra+Pradesh%2C+India",
  "management": "Government",
  "recognised_by": "Dental Council of India",
  "status": "Recognised",
  "total_intake": "50",
  "total_faculty": "100",
  "location_type": "Urban",
  "gender": "Co-education",
  "total_area": "10 acres",
  "hospital_associated": "RIMS Hospital",
  "hospital_distance": "Within the campus",
  "nearest_airport": "Vijayawada Airport (120 km)",
  "nearest_railway": "Kadapa Railway Station (5 km)",
  "road_connectivity": "Good",
  "ug_course

In [ ]:
college_3 = await get_accurate_college_profile(
    "Govt. Dental College, Dibrugarh",
    "Assam"
)
print(json.dumps(college_3, indent=2))
college_records.append(college_3)

Starting audit: Govt. Dental College, Dibrugarh


[FETCH]... ↓ https://dciindia.gov.in/Admin/NewsArchives/N_120824125605149.pdf                                     |
✓ | ⏱: 4.08s 

[SCRAPE].. ◆ https://dciindia.gov.in/Admin/NewsArchives/N_120824125605149.pdf                                     |
✓ | ⏱: 0.01s 

[COMPLETE] ● https://dciindia.gov.in/Admin/NewsArchives/N_120824125605149.pdf                                     |
✓ | ⏱: 4.12s 

[FETCH]... ↓ https://edufever.in/colleges/government-dental-college-dibrugarh/                                    |
✓ | ⏱: 6.87s 

...

[COMPLETE] ● https://www.edufever.com/govt-dental-college-dibrugarh/                                              |
✓ | ⏱: 15.21s 

[FETCH]... ↓ https://rmgoe.org/study-india/govt-dental-college-dibrugarh                                          |
✓ | ⏱: 16.67s 

[SCRAPE].. ◆ https://rmgoe.org/study-india/govt-dental-college-dibrugarh                                          |
✓ | ⏱: 3.12s 

[COMPLETE] ● https://rmgoe.org/study-india/govt-dental-college-dibrugarh                                          |
✓ | ⏱: 19.87s 

Groq auditing: Govt. Dental College, Dibrugarh
Audit complete: Govt. Dental College, Dibrugarh
{
  "name": "Govt. Dental College, Dibrugarh",
  "state": "Assam",
  "address": "FWQV+JX9, I Ln, Dibrugarh, Assam 786002",
  "google_rating": 4.5,
  "google_reviews": "53 Google reviews",
  "management": "Government",
  "recognised_by": "Dental Council of India",
  "status": "Recognised",
  "total_intake": "50",
  "total_faculty": "100",
  "location_type": "Urban",
  "gender": "Co-education",
  "total_area": "10 acres",
  "hospital_associated": "Yes",
  "hospital_distance": "1 km",
  "nearest_airport": "Dibrugarh Airport, 5 km",
  "nearest_railway": "Dibrugarh Railway Station, 3 km",
  "road_connectivity": "Good",
  "ug_courses": 1,
  "pg_courses": 0,
  "university": "Dibrugarh University",
  "estd": 1984,
  "short_overview": "Govt. Dental College, Dibrugarh is a government dental college located in Dibrugarh, Assam. It was established in 1984 and is recognized by the Dental Council of India.

In [ ]:
college_4 = await get_accurate_college_profile(
    "Govt. Dental College, Silchar",
    "Assam"
)
print(json.dumps(college_4, indent=2))
college_records.append(college_4)

Starting audit: Govt. Dental College, Silchar


[FETCH]... ↓ https://en.wikipedia.org/wiki/Government_Dental_College,_Silchar                                     |
✓ | ⏱: 6.01s 

[SCRAPE].. ◆ https://en.wikipedia.org/wiki/Government_Dental_College,_Silchar                                     |
✓ | ⏱: 0.88s 

[COMPLETE] ● https://en.wikipedia.org/wiki/Government_Dental_College,_Silchar                                     |
✓ | ⏱: 6.97s 

[FETCH]... ↓ https://www.facebook.com/100054397754928/posts/a...ducted-at-govt-dental-college-/1065438491946066/  |
✓ | ⏱: 9.01s 

...

[COMPLETE] ● https://www.schoolmykids.com/college/government-dental-college-silchar-assam                         |
✓ | ⏱: 16.73s 

[FETCH]... ↓ https://www.facebook.com/p/Government-Dental-College-Silchar-100054397754928/                        |
✓ | ⏱: 18.09s 

[SCRAPE].. ◆ https://www.facebook.com/p/Government-Dental-College-Silchar-100054397754928/                        |
✓ | ⏱: 0.37s 

[COMPLETE] ● https://www.facebook.com/p/Government-Dental-College-Silchar-100054397754928/                        |
✓ | ⏱: 18.54s 

Groq auditing: Govt. Dental College, Silchar
Audit complete: Govt. Dental College, Silchar
{
  "name": "Govt. Dental College, Silchar",
  "state": "Assam",
  "address": "QQFR+39C, Masimpur, Silchar, Silcoorie Grant, Assam 788014",
  "google_rating": 3.8,
  "google_reviews": "51 Google reviews",
  "management": "Government",
  "recognised_by": "Dental Council of India",
  "status": "Recognised",
  "total_intake": "50 per year",
  "total_faculty": "Unknown",
  "location_type": "Urban",
  "gender": "Co-education",
  "total_area": "Unknown",
  "hospital_associated": "Yes",
  "hospital_distance": "Unknown",
  "nearest_airport": "Silchar Airport, 5 km",
  "nearest_railway": "Silchar Railway Station, 5 km",
  "road_connectivity": "Good",
  "ug_courses": 1,
  "pg_courses": 0,
  "university": "Srimanta Sankaradeva University of Health Sciences",
  "estd": 2018,
  "short_overview": "Government Dental College, Silchar is a government dental college of Assam. It was established in 2018 by Governme

In [ ]:
college_5 = await get_accurate_college_profile(
    "Regional Dental College, Guwahati",
    "Assam"
)

print(json.dumps(college_5, indent=2))
college_records.append(college_5)

Starting audit: Regional Dental College, Guwahati


[FETCH]... ↓ https://en.wikipedia.org/wiki/Regional_Dental_College,_Guwahati                                      |
✓ | ⏱: 11.51s 

[SCRAPE].. ◆ https://en.wikipedia.org/wiki/Regional_Dental_College,_Guwahati                                      |
✓ | ⏱: 1.34s 

[COMPLETE] ● https://en.wikipedia.org/wiki/Regional_Dental_College,_Guwahati                                      |
✓ | ⏱: 13.05s 

[FETCH]... ↓ https://rdc.edu.in/                                                                                  |
✓ | ⏱: 13.63s 

...

[COMPLETE] ● https://www.facebook.com/groups/643524902972581/posts/1747515405906853/                              |
✓ | ⏱: 22.39s 

[FETCH]... ↓ https://www.facebook.com/rdcguwahati/                                                                |
✓ | ⏱: 23.13s 

[SCRAPE].. ◆ https://www.facebook.com/rdcguwahati/                                                                |
✓ | ⏱: 0.08s 

[COMPLETE] ● https://www.facebook.com/rdcguwahati/                                                                |
✓ | ⏱: 23.23s 

Groq auditing: Regional Dental College, Guwahati
Audit complete: Regional Dental College, Guwahati
{
  "name": "Regional Dental College, Guwahati",
  "state": "Assam",
  "address": "G.M.C.H. Complex, Bhangagarh, Guwahati, Assam 781032",
  "google_rating": 3.8,
  "google_reviews": "181 Google reviews",
  "management": "Government",
  "recognised_by": "Dental Council of India",
  "status": "Recognised",
  "total_intake": "50 (BDS) and 10 (MDS)",
  "total_faculty": "Not specified",
  "location_type": "Urban",
  "gender": "Co-education",
  "total_area": "Not specified",
  "hospital_associated": "Yes",
  "hospital_distance": "On campus",
  "nearest_airport": "Guwahati International Airport (10 km)",
  "nearest_railway": "Guwahati Railway Station (10 km)",
  "road_connectivity": "Good",
  "ug_courses": 1,
  "pg_courses": 5,
  "university": "Srimanta Sankaradeva University of Health Sciences",
  "estd": 1982,
  "short_overview": "Regional Dental College, Guwahati is a government dental colleg

In [ ]:
college_6 = await get_accurate_college_profile(
    "Government Dental College & Hospital, Nalanda",
    "Bihar"
)
print(json.dumps(college_6, indent=2))
college_records.append(college_6)

In [ ]:
college_7 = await get_accurate_college_profile(
    "Patna Dental College & Hospital, Patna",
    "Bihar"
)
print(json.dumps(college_7, indent=2))
college_records.append(college_7)

In [ ]:
college_8 = await get_accurate_college_profile(
    "Dr. Harvansh Singh Judge Institute of Dental Sciences & Hospital, Chandigarh",
    "Chandigarh"
)
print(json.dumps(college_8, indent=2))
college_records.append(college_8)

In [ ]:
college_9 = await get_accurate_college_profile(
    "Govt. Dental College, Raipur",
    "Chhattisgarh"
)
print(json.dumps(college_9, indent=2))
college_records.append(college_9)

In [ ]:
college_10 = await get_accurate_college_profile(
    "ESIC Dental College, Rohini",
    "New Delhi"
)
print(json.dumps(college_10, indent=2))
college_records.append(college_10)

In [ ]:
college_11 = await get_accurate_college_profile(
    "Faculty of Dentistry, Jamia Millia Islamia",
    "New Delhi"
)
print(json.dumps(college_11, indent=2))
college_records.append(college_11)

In [ ]:
college_12 = await get_accurate_college_profile(
    "Maulana Azad Dental College & Hospital",
    "New Delhi"
)
print(json.dumps(college_12, indent=2))
college_records.append(college_12)

In [ ]:
college_13 = await get_accurate_college_profile(
    "Goa Dental College & Hospital",
    "Goa"
)
print(json.dumps(college_13, indent=2))
college_records.append(college_13)

In [ ]:
college_14 = await get_accurate_college_profile(
    "AMC Dental College, Ahmedabad",
    "Gujarat"
)
print(json.dumps(college_14, indent=2))
college_records.append(college_14)

In [ ]:
college_15 = await get_accurate_college_profile(
    "Govt. Dental College & Hospital, Ahmedabad",
    "Gujarat"
)
print(json.dumps(college_15, indent=2))
college_records.append(college_15)

In [ ]:
college_16 = await get_accurate_college_profile(
    "Govt. Dental College Hospital, Jamnagar",
    "Gujarat"
)
print(json.dumps(college_16, indent=2))
college_records.append(college_16)

In [ ]:
college_17 = await get_accurate_college_profile(
    "Siddhpur Dental College, Patan",
    "Gujarat"
)
print(json.dumps(college_17, indent=2))
college_records.append(college_17)

In [ ]:
college_18 = await get_accurate_college_profile(
    "Post Graduate Institute of Dental Sciences, Rohtak",
    "Haryana"
)
print(json.dumps(college_18, indent=2))
college_records.append(college_18)

In [ ]:
college_19 = await get_accurate_college_profile(
    "H.P. Govt. Dental College & Hospital, Shimla",
    "Himachal Pradesh"
)
print(json.dumps(college_19, indent=2))
college_records.append(college_19)

In [ ]:
college_20 = await get_accurate_college_profile(
    "Govt. Dental College, Srinagar",
    "Jammu & Kashmir"
)
print(json.dumps(college_20, indent=2))
college_records.append(college_20)

In [ ]:
college_21 = await get_accurate_college_profile(
    "Indira Gandhi Government Dental College, Jammu",
    "Jammu & Kashmir"
)
print(json.dumps(college_21, indent=2))
college_records.append(college_21)

In [ ]:
college_22 = await get_accurate_college_profile(
    "Dental Institute, RIMS, Ranchi",
    "Jharkhand"
)
print(json.dumps(college_22, indent=2))
college_records.append(college_22)

In [ ]:
college_23 = await get_accurate_college_profile(
    "ESIC Dental College, Gulbarga",
    "Karnataka"
)
print(json.dumps(college_23, indent=2))
college_records.append(college_23)

In [ ]:
college_24 = await get_accurate_college_profile(
    "Govt. Dental College & Research Institute, Bellary",
    "Karnataka"
)
print(json.dumps(college_24, indent=2))
college_records.append(college_24)

In [ ]:
college_25 = await get_accurate_college_profile(
    "Govt. Dental College, Bangalore",
    "Karnataka"
)
print(json.dumps(college_25, indent=2))
college_records.append(college_25)

In [ ]:
college_26 = await get_accurate_college_profile(
    "Govt. Dental College, Alappuzha",
    "Kerala"
)
print(json.dumps(college_26, indent=2))
college_records.append(college_26)

In [ ]:
college_27 = await get_accurate_college_profile(
    "Govt. Dental College, Kottayam",
    "Kerala"
)
print(json.dumps(college_27, indent=2))
college_records.append(college_27)

In [ ]:
college_28 = await get_accurate_college_profile(
    "Govt. Dental College, Kozhikode",
    "Kerala"
)
print(json.dumps(college_28, indent=2))
college_records.append(college_28)

In [ ]:
college_29 = await get_accurate_college_profile(
    "Govt. Dental College, Thrissur",
    "Kerala"
)
print(json.dumps(college_29, indent=2))
college_records.append(college_29)

In [ ]:
college_30 = await get_accurate_college_profile(
    "Govt. Dental College, Trivandrum",
    "Kerala"
)
print(json.dumps(college_30, indent=2))
college_records.append(college_30)

In [ ]:
college_31 = await get_accurate_college_profile(
    "Pariyaram Dental College, Academy of Medical Sciences, Kannur",
    "Kerala"
)
print(json.dumps(college_31, indent=2))
college_records.append(college_31)

In [ ]:
college_32 = await get_accurate_college_profile(
    "College of Dentistry, Indore",
    "Madhya Pradesh"
)
print(json.dumps(college_32, indent=2))
college_records.append(college_32)

In [ ]:
college_33 = await get_accurate_college_profile(
    "Govt. Dental College & Hospital, Aurangabad",
    "Maharashtra"
)
print(json.dumps(college_33, indent=2))
college_records.append(college_33)

In [ ]:
college_34 = await get_accurate_college_profile(
    "Govt. Dental College & Hospital, Mumbai",
    "Maharashtra"
)
print(json.dumps(college_34, indent=2))
college_records.append(college_34)

In [ ]:
college_35 = await get_accurate_college_profile(
    "Govt. Dental College & Hospital, Nagpur",
    "Maharashtra"
)
print(json.dumps(college_35, indent=2))
college_records.append(college_35)

In [ ]:
college_36 = await get_accurate_college_profile(
    "Nair Hospital Dental College, Mumbai",
    "Maharashtra"
)
print(json.dumps(college_36, indent=2))
college_records.append(college_36)

In [ ]:
college_37 = await get_accurate_college_profile(
    "Dental College, Jawaharlal Nehru Institute of Medical Sciences, Imphal",
    "Manipur"
)
print(json.dumps(college_37, indent=2))
college_records.append(college_37)

In [ ]:
college_38 = await get_accurate_college_profile(
    "Dental College, RIMS, Imphal",
    "Manipur"
)
print(json.dumps(college_38, indent=2))
college_records.append(college_38)

In [ ]:
college_39 = await get_accurate_college_profile(
    "S.C.B. Dental College & Hospital, Cuttack",
    "Odisha"
)
print(json.dumps(college_39, indent=2))
college_records.append(college_39)

In [ ]:
college_40 = await get_accurate_college_profile(
    "Mahatma Gandhi Post Graduate Institute of Dental Sciences, Puducherry",
    "Puducherry"
)
print(json.dumps(college_40, indent=2))
college_records.append(college_40)

In [ ]:
college_41 = await get_accurate_college_profile(
    "Govt. Dental College & Hospital, Patiala",
    "Punjab"
)
print(json.dumps(college_41, indent=2))
college_records.append(college_41)

In [ ]:
college_42 = await get_accurate_college_profile(
    "Pb. Govt. Dental College & Hospital, Amritsar",
    "Punjab"
)
print(json.dumps(college_42, indent=2))
college_records.append(college_42)

In [ ]:
college_43 = await get_accurate_college_profile(
    "Government Dental College, Jodhpur",
    "Rajasthan"
)
print(json.dumps(college_43, indent=2))
college_records.append(college_43)

In [ ]:
college_44 = await get_accurate_college_profile(
    "Govt. Dental College & Hospital, Jaipur",
    "Rajasthan"
)
print(json.dumps(college_44, indent=2))
college_records.append(college_44)

In [ ]:
college_45 = await get_accurate_college_profile(
    "Government Dental College, Pudukkottai",
    "Tamil Nadu"
)
print(json.dumps(college_45, indent=2))
college_records.append(college_45)

In [ ]:
college_46 = await get_accurate_college_profile(
    "Govt. Dental College & Hospital, Cuddalore",
    "Tamil Nadu"
)
print(json.dumps(college_46, indent=2))
college_records.append(college_46)

In [ ]:
college_47 = await get_accurate_college_profile(
    "Tamil Nadu Government Dental College & Hospital, Chennai",
    "Tamil Nadu"
)
print(json.dumps(college_47, indent=2))
college_records.append(college_47)

In [ ]:
college_48 = await get_accurate_college_profile(
    "Government Dental College and Hospital, Hyderabad",
    "Telangana"
)
print(json.dumps(college_48, indent=2))
college_records.append(college_48)

In [ ]:
college_49 = await get_accurate_college_profile(
    "Agartala Government Dental College and IGM Hospital",
    "Tripura"
)
print(json.dumps(college_49, indent=2))
college_records.append(college_49)

In [ ]:
college_50 = await get_accurate_college_profile(
    "Dr. Ziauddin Ahmad Dental College, Aligarh",
    "Uttar Pradesh"
)
print(json.dumps(college_50, indent=2))
college_records.append(college_50)

In [ ]:
college_51 = await get_accurate_college_profile(
    "Faculty of Dental Sciences, Banaras Hindu University, Varanasi",
    "Uttar Pradesh"
)
print(json.dumps(college_51, indent=2))
college_records.append(college_51)

In [ ]:
college_52 = await get_accurate_college_profile(
    "Faculty of Dental Sciences, Lucknow",
    "Uttar Pradesh"
)
print(json.dumps(college_52, indent=2))
college_records.append(college_52)

In [ ]:
college_53 = await get_accurate_college_profile(
    "Burdwan Dental College, Rajbati",
    "West Bengal"
)
print(json.dumps(college_53, indent=2))
college_records.append(college_53)

In [ ]:
college_54 = await get_accurate_college_profile(
    "Dr. R. Ahmed Dental College & Hospital, Calcutta",
    "West Bengal"
)
print(json.dumps(college_54, indent=2))
college_records.append(college_54)

In [ ]:
college_55 = await get_accurate_college_profile(
    "The North Bengal Dental College, Sushratangar",
    "West Bengal"
)
print(json.dumps(college_55, indent=2))
college_records.append(college_55)

In [ ]:
college_56 = await get_accurate_college_profile(
    "Govt. Dental College & Hospital, Jalgaon",
    "Maharashtra"
)
print(json.dumps(college_56, indent=2))
college_records.append(college_56)

In [ ]:
df_colleges = pd.DataFrame(college_records)
df_colleges


In [ ]:
df_colleges.to_csv("government_dental_colleges_india.csv", index=False)
